In [1]:
import sys
import time
import tensorflow as tf
import numpy as np
sys.path.insert(0, '../')
%load_ext autoreload
%autoreload 2

In [2]:
from seqmodel.bunch import Bunch
from seqmodel import model
from seqmodel import data

In [3]:
tf.reset_default_graph()
model_opt = model.seq2seq_model.BasicSeq2SeqModel.default_opt()
model_opt.embedding.decoder_dim = 10
model_opt.embedding.encoder_dim = 10
model_opt.decoder.rnn_opt.rnn_cell.cell_opt.num_units = 10
model_opt.encoder.rnn_opt.rnn_cell.cell_opt.num_units = 10
model_opt.decoder.share.encoder_rnn_params = True
model_opt.decoder.share.encoder_embedding = True
model_opt.decoder.share.logit_weight_tying = True
model_fn = model.seq2seq_model.BasicSeq2SeqModel(model_opt)
seq2seq_model = model_fn(is_training=True)
optimizer = tf.train.AdamOptimizer()
g_v_pairs =  optimizer.compute_gradients(seq2seq_model.training_loss, tf.trainable_variables())
train_op = optimizer.apply_gradients(g_v_pairs)
for v in tf.trainable_variables():
    print("{}, {}".format(v.name, v.get_shape()))

model/shared_embedding:0, (15, 10)
model/shared_rnn/rnn/basic_lstm_cell/weights:0, (20, 40)
model/shared_rnn/rnn/basic_lstm_cell/biases:0, (40,)
model/decoder_rnn/logit_b:0, (15,)


In [4]:
vocab = data.Vocabulary.from_vocab_file('../data/tiny_copy/vocab.txt')
valid_iter = data.Seq2SeqIterator(vocab, vocab)
valid_iter.initialize('../data/tiny_copy/valid.txt')
valid_iter.init_batch(4)

In [5]:
sess_config = tf.ConfigProto(device_count={'GPU': 0})
sess = tf.Session(config = sess_config)
sess.run(tf.global_variables_initializer())
batch = valid_iter.next_batch()
state = None

In [6]:
prediction, state, info = seq2seq_model.predict(sess, batch.features)

In [7]:
eval_loss, state, info = seq2seq_model.evaluate(sess, batch)

In [8]:
eval_loss, train_loss, state, info = seq2seq_model.train(sess, batch, train_op)

In [9]:
eval_loss

2.6866577

In [10]:
train_loss

80.599731

In [70]:
tf.reset_default_graph()
gx = tf.TensorArray(dtype=tf.int32, size=10)

def cond(i, _):
    return i < 10

def body(i, gx_):
    gx_ = gx_.write(i, i * 10)
    return i + 1, gx_

a, b = tf.while_loop(cond=cond, body=body,
                     loop_vars=[tf.constant(0, tf.int32), gx])
o = b.stack()

In [71]:
sess_config = tf.ConfigProto(device_count={'GPU': 0})
sess = tf.Session(config=sess_config)
sess.run(tf.global_variables_initializer())
sess.run(o)

array([ 0, 10, 20, 30, 40, 50, 60, 70, 80, 90], dtype=int32)